In [3]:
import os
import io
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from web3 import Web3

## Example of storing File on blockchain an then retrieving data from it

In [2]:
import requests
# Path to CSV file
file_path = 'Data\signals.csv'

# IPFS HTTP API endpoint
url = 'http://127.0.0.1:5001/api/v0/add'

# Upload the file
with open(file_path, 'rb') as f:
    response = requests.post(url, files={'file': f})


# Get the CID from the response
cid = response.json()['Hash']
filename = response.json()['Name']
print(f'File uploaded to IPFS with CID: {cid}, with name {filename}')




File uploaded to IPFS with CID: Qma94VFefdqoaMq3QZGsh2qPn7p3XMkE9N4MszRwCanNCW, with name signals.csv


In [1]:
from web3 import Web3

# Connect to Ganache
ganache_url = 'http://127.0.0.1:8545'
w3 = Web3(Web3.HTTPProvider(ganache_url))

# Ensure connection is successful
assert w3.is_connected()

# Address of the deployed contract
contract_address = '0xB7Aa04Bce00169Ea7977969f57F698d5849A035f'

# ABI of the deployed contract
contract_abi = [
   {
       "inputs": [
           {
               "internalType": "string",
               "name": "_cid",
               "type": "string"
           }
       ],
       "name": "setCID",
       "outputs": [],
       "stateMutability": "nonpayable",
       "type": "function"
   },
   {
       "inputs": [],
       "name": "getCID",
       "outputs": [
           {
               "internalType": "string",
               "name": "",
               "type": "string"
           }
       ],
       "stateMutability": "view",
       "type": "function"
   }
]

# Create contract instance
contract = w3.eth.contract(address=contract_address, abi=contract_abi)

# Replace with your actual private key and account address
private_key = "0x006b6a54b8d97432c7c0ad7e89e338fdfb3bfe170574e6306dd4b9285a829169"
account = "0xF38dd7134610bC879bBdE056420F40f221cDF389"

# Check account balance
balance = w3.eth.get_balance(account)
print(f'Account balance: {w3.from_wei(balance, "ether")} Ether')

# Ensure the account has sufficient funds
required_balance = w3.to_wei(0.1, 'ether')  # Adjust this as needed

if balance < required_balance:
    print(f'Insufficient funds. Please send at least {w3.from_wei(required_balance, "ether")} Ether to the account.')

# Set the CID in the contract
def set_cid(cid):
   nonce = w3.eth.get_transaction_count(account)
   txn = contract.functions.setCID(cid).build_transaction({
       'chainId': 1337,  # Ganache chain ID
       'gas': 6721975,
       'gasPrice': w3.to_wei('20', 'gwei'),
       'nonce': nonce,
   })

   signed_txn = w3.eth.account.sign_transaction(txn, private_key=private_key)
   tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
   tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
   return tx_receipt

# Ensure account has sufficient funds before proceeding
if balance >= required_balance:
    # Example CID obtained from IPFS upload
    cid = 'Qma94VFefdqoaMq3QZGsh2qPn7p3XMkE9N4MszRwCanNCW'
    # Set CID
    receipt = set_cid(cid)
    # Retrieve the CID from the contract
    def get_cid():
        return contract.functions.getCID().call()

    # Get CID
    stored_cid = get_cid()
    print(f'Stored CID: {stored_cid}')
else:
    print('Please fund the account and try again.')

AssertionError: 

In [43]:
stored_cid = get_cid()
gateway_url = f'http://127.0.0.1:8081/ipfs/{stored_cid}'

# Retrieve the file content from IPFS
response = requests.get(gateway_url)

data_signals = response.content.decode('utf-8')
df = pd.read_csv(io.StringIO(data_signals))

# Display the DataFrame
print(df.head())

   Unnamed: 0   X1   X2   X3   X4   X5   X6   X7   X8   X9  ...  X170  X171  \
0  X21.V1.791  135  190  229  223  192  125   55   -9  -33  ...   -17   -15   
1  X15.V1.924  386  382  356  331  320  315  307  272  244  ...   164   150   
2     X8.V1.1  -32  -39  -47  -37  -32  -36  -57  -73  -85  ...    57    64   
3   X16.V1.60 -105 -101  -96  -92  -89  -95 -102 -100  -87  ...   -82   -81   
4   X20.V1.54   -9  -65  -98 -102  -78  -48  -16    0  -21  ...     4     2   

   X172  X173  X174  X175  X176  X177  X178  y  
0   -31   -77  -103  -127  -116   -83   -51  4  
1   146   152   157   156   154   143   129  1  
2    48    19   -12   -30   -35   -35   -36  5  
3   -80   -77   -85   -77   -72   -69   -65  5  
4   -12   -32   -41   -65   -83   -89   -73  5  

[5 rows x 180 columns]


In [44]:
# CID X_train
cid2 = 'QmeevmCbbqcETE6hEyBvExd3mcDsJ8DZvBBatv8mtbSUzX' 
gateway_url2 = f'http://127.0.0.1:8081/ipfs/{cid2}'
response = requests.get(gateway_url2)
x_train = pd.read_csv(io.StringIO(response.content.decode('utf-8')))

#CID of x_test
cid3 = "QmWibTftNUmUDoEBhdhGuqdVGKP92r9HkjaijUY7z7B4kP"
gateway_url3 = f'http://127.0.0.1:8081/ipfs/{cid3}'
response = requests.get(gateway_url3)
x_test =pd.read_csv(io.StringIO(response.content.decode('utf-8')))
#CID of y_train
cid4 = "QmPPhHJRYguRpKb8wUJrzturGDnZRYMHddxerYbam6Qdxz"
gateway_url4 = f'http://127.0.0.1:8081/ipfs/{cid4}'
response = requests.get(gateway_url4)
y_train = pd.read_csv(io.StringIO(response.content.decode('utf-8')))

#CID of y_test
cid5 = "QmVpX6VX3Kfkpg53YgbfQHKtMQNWb5SVEuvCMqxLpisc3d"
gateway_url5 = f'http://127.0.0.1:8081/ipfs/{cid5}'
response = requests.get(gateway_url5)
y_test = pd.read_csv(io.StringIO(response.content.decode('utf-8')))

In [45]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((42550, 178), (1150, 178), (42550, 1), (1150, 1))

In [17]:
x_train = pd.read_csv('data/x_train.csv')
x_test = pd.read_csv('data/x_test.csv')
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')


x_train.shape, x_test.shape, y_train.shape, y_test.shape

((42550, 178), (1150, 178), (42550, 1), (1150, 1))

In [4]:
X_reshaped = np.array([x_train.iloc[i, :].values for i in range(len(x_train))])

output_folder = 'person_images_train'
os.makedirs(output_folder, exist_ok=True)

for i, person_data in enumerate(tqdm(X_reshaped, desc='Processing images')):
    person_matrix = person_data.reshape((1, 178))
    person_matrix = person_matrix / person_matrix.max()
    plt.figure(figsize=(8, 4))
    plt.imshow(person_matrix, cmap='viridis', interpolation='nearest', aspect='auto')
    plt.axis('off')
    image_path = os.path.join(output_folder, f'reading_{i}.png')
    plt.imsave(image_path, person_matrix, cmap='viridis')
    plt.close()

print(f"Images are saved in the folder: {output_folder}")

Processing images:   4%|▍         | 1733/42550 [00:30<12:27, 54.58it/s]C:\Users\Mahmoud\AppData\Local\Temp\ipykernel_13644\1668597394.py:8: RuntimeWarning: divide by zero encountered in divide
  person_matrix = person_matrix / person_matrix.max()
C:\Users\Mahmoud\AppData\Local\Temp\ipykernel_13644\1668597394.py:8: RuntimeWarning: invalid value encountered in divide
  person_matrix = person_matrix / person_matrix.max()
Processing images: 100%|██████████| 42550/42550 [20:48<00:00, 34.07it/s]   

Images are saved in the folder: person_images_train


In [4]:
X_reshaped = np.array([x_test.iloc[i, :].values for i in range(len(x_test))])

output_folder = 'person_images_test'
os.makedirs(output_folder, exist_ok=True)

for i, person_data in enumerate(tqdm(X_reshaped, desc='Processing images')):
    person_matrix = person_data.reshape((1, 178))
    person_matrix = person_matrix / person_matrix.max()
    plt.figure(figsize=(8, 4))
    plt.imshow(person_matrix, cmap='viridis', interpolation='nearest', aspect='auto')
    plt.axis('off')
    image_path = os.path.join(output_folder, f'reading_{i+1}.png')
    plt.imsave(image_path, person_matrix, cmap='viridis')
    plt.close()

print(f"Images are saved in the folder: {output_folder}")

Processing images:   0%|          | 0/1150 [00:00<?, ?it/s]

Processing images:  36%|███▌      | 415/1150 [00:01<00:03, 218.99it/s]/var/folders/78/20zm8p5d34g6v9kz5nf2tbnm0000gn/T/ipykernel_27969/3224925415.py:8: RuntimeWarning: divide by zero encountered in divide
  person_matrix = person_matrix / person_matrix.max()
/var/folders/78/20zm8p5d34g6v9kz5nf2tbnm0000gn/T/ipykernel_27969/3224925415.py:8: RuntimeWarning: invalid value encountered in divide
  person_matrix = person_matrix / person_matrix.max()
Processing images: 100%|██████████| 1150/1150 [00:05<00:00, 202.13it/s]

Images are saved in the folder: person_images_test
